In [20]:
import pandas as pd
from ast import literal_eval
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

df = pd.read_csv('tools-for-thought.csv')
sample = df.sample(100)

In [21]:
from ast import literal_eval

sample['media'] = df['attachments.media'].fillna('[]').apply(literal_eval).apply(lambda x: [{'type': i['type']} for i in x if type(i) == dict and "type" in i.keys()])
sample['entities.annotations'] = sample['entities.annotations'].fillna('[]').apply(literal_eval).apply(lambda x: [{"type": i['type'], "text": i['normalized_text']} for i in x])
sample['context.annotations'] = sample["context_annotations"].fillna('[]').apply(literal_eval).apply(lambda x: [{"domain": i['domain']['name'], "entity": i['entity']['name']} for i in x])

In [22]:
# mark standalone, reply, rt
sample['rt'] = sample["referenced_tweets.retweeted.id"].isna().apply(lambda x: not x)
sample['reply'] = sample["referenced_tweets.replied_to.id"].isna().apply(lambda x: not x)
sample['quote'] = sample["referenced_tweets.quoted.id"].isna().apply(lambda x: not x)
sample['standalone'] = df[["referenced_tweets.replied_to.id",	"referenced_tweets.retweeted.id", "referenced_tweets.quoted.id"]].isna().sum(axis=1).apply(lambda x: True if x == 3 else False)

# rename author.name to name
sample = sample.rename(columns={'author.name': 'name', 'author.username': 'username', 'author.description': 'description'})

# find all columns starting with 'public metrics'
names = [col for col in df.columns if col.startswith('public_metrics')]
# remove 
new_names = [col.split('.')[1] for col in names]
sample = sample.rename(columns=dict(zip(names, new_names)))

In [23]:
# Convert to desired schema

sample['id'] = sample['id'].astype(str)
sample['author'] = sample[['name', 'username', 'description']].to_dict(orient='records')
sample['html'] = sample['text'].astype(str)
sample['public_metrics'] = sample[new_names].to_dict(orient='records')

sample = sample[['id', 'author', 'html', 'created_at', 'public_metrics', 'standalone', 'rt', 'reply', 'quote', 'geo.country', 'entities.annotations', 'context_annotations', 'media']]

sample.to_json('sample.json', orient='records')